Main Body Architecture Consists of 2 parts
    1. Convolutional Layers - Use kernels to extract useful features
    2. Fully Connected (FC) Layers  - using features to classify images


In [ ]:
print("Hello World")